# bedRMod format

```
Col	BED Field	Type	Value	Brief description
1	chrom	String	[[:alnum:]_]{1,255}	Chromosome name
2	chromStart	Int	[0, 264 − 1]	Feature start position
3	chromEnd	Int	[0, 264 − 1]	Feature end position
4	name	String	[[:alnum:]_]{1,255}	Modification name (MODOMICS short name)
5	score	Int	[0, 1000]	Modification confidence scaled from 0 - 1000
6	strand	String	[-+.]	Feature strand
7	thickStart	Int	[0, 264 − 1]	Thick start position
8	thickEnd	Int	[0, 264 − 1]	Thick end position
9	itemRgb	Int,Int,Int	([0, 255], [0, 255], [0, 255]) | 0	Display color
10	coverage	Int	[0, 2^64 − 1]	Number of reads at this position
11	frequency	Int	[0, 100]	Percentage of modified reads at thi
12	refBase	Char	[A, U, G, C, N]	Reference base at this position
```

In [20]:
#%matplotlib inline
import joblib
import csv, gzip, os, matplotlib.pyplot as plt, numpy as np, pandas as pd, pysam, scipy, sys, joblib, seaborn as sns#; sns.set()
from glob import glob
from collections import Counter
from scipy.stats import stats
from datetime import datetime
from multiprocessing import Pool

%load_ext autoreload
%autoreload 2
# import modules from src dir
sys.path.insert(0, "/home/lpryszcz/src/nanoRMS3/src")
#import eif_new as iso_new
from predict_mods import bam2data, load_data, load_data_single, get_KS, get_revcomp, \
    get_calls, get_regions, get_covered_regions, get_significant_mers, \
    get_classifier, get_freq, GradientBoostingClassifier, random_state, \
    fasta2bases
# make sure plots are inline (modPhred uses agg)
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
name = "Y"
color = np.array([255, 0, 0], dtype="int") # RGB

rna = True
n = 3
nn = 1
mapq = 15
minReads = 1
maxReads = 100000

dt_shift = 10 # expected shift between center of the pore and motor protein in bases

features = ["si", "mp", "dt0", "dt%s"%dt_shift] + ["t%s"%b for b in "ACGT"] # here we don't need TR as TA/C/G/T will capture it all right
feature_names = ["%s_%s"%(f.upper(), i) for f in features for i in range(-nn, nn+1)]

boi, k, n_features = "T", 2*n+1, len(feature_names)
modelsfn = "3a.models.%s.%s_mers.%s_features.lzma"%(boi, k, n_features)
mer2clf = joblib.load(modelsfn); len(mer2clf)

243

In [38]:
",".join(map(str, np.round(color*0.5).astype('int')))

'128,0,0'

In [40]:
fasta = "/home/lpryszcz/cluster/rna_mods/RMaP_challenge/Manja_Merz/3a/reference/psU_template.fa" # reference FastA
faidx = pysam.FastaFile(fasta)

bams = ["/home/lpryszcz/cluster/rna_mods/RMaP_challenge/Manja_Merz/3a/fast5/testing.bam", ]
sams = [pysam.AlignmentFile(bam) for bam in bams]
outs = [open(bam+".bed", "wt") for bam in bams]

only_forward = True
max_frac_mod_per_read = 0.3
line = "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n"
out = sys.stdout
for ref, reflen in zip(faidx.references, faidx.lengths):
    start, end = 0, reflen
    if start<nn: start=nn
    parsers = [bam2data(sam, ref, start-nn, end+nn+1, rna, nn, features, mapq=mapq, 
                        maxDepth=maxReads*3, verbose=False, max_frac_mod_per_read=max_frac_mod_per_read)
               for sam in sams]
    refparser = fasta2bases(faidx, ref, start, end, n=n)
    for ((pos, _, strand, refbase, mer), *calls) in zip(refparser, *parsers):
        if mer not in mer2clf: continue
        # here we'd need to flip -1, 0, +1 for strand -
        if strand=="-":
            if only_forward: continue
            calls = [np.flip(c, axis=2) for c in calls]
        sample2data = [np.hstack(c) for c in calls]
        refBase = mer[n]
        s, e = pos, pos+1
        for out, X in zip(outs, sample2data):
            mod_freq = y_pred.mean()
            y_pred = mer2clf[mer].predict(X)
            coverage = len(X)
            frequency = int(np.round(100*mod_freq))
            # this could be better
            score = int(round(max_score * mod_freq)) # 0-1000 scaled
            # scale color
            c = ",".join(map(str, np.round(color*mod_freq).astype('int')))
            out.write(line%(ref, s, e, name, score, strand, s, e, c, coverage, frequency, refBase))

for out in outs: 
    out.close()